# Required modules

In [ ]:
# import system modules
import pandas as pd
import ee
import os
import sys

In [ ]:
# import user modules
sys.dont_write_bytecode = True
sys.path.append(os.path.abspath('../../'))

from modules.Collection import getCollection
from modules.BandNames import getBandNames
from modules.CloudAndShadowMask import getMasks
from modules.SMA_NDFI import *

In [ ]:
pd.set_option('mode.chained_assignment', None)

In [ ]:
# intialize earth engine credentials
ee.Initialize()

# Random Forest parameterization

In [ ]:
# random forest params
RF_PARAMS = {
    'numberOfTrees': 50,
    'variablesPerSplit': 4,
    'minLeafPopulation': 25
}

# define the feature space
FEAT_SPACE_BANDS = ["gv", "gvs", "soil", "npv", "shade", "ndfi", "csfi"]

# Sampling parameters

In [ ]:
# samples version
V_SAMPLES = '5'

samplesBaseName = 'projects/imazon-simex/LULC/SAMPLES/COLLECTION6/TRAINED/samples-amazon-collection-6-{}-{}'

# number of samples
N_SAMPLES = 10000

# percentage of samples within the sample universe
P_SAMPLES = {"global": 0.30, "regional": 0.70}

# minimum number of samples per classe
dfMinSamples = pd.DataFrame([
    {'class':  3, 'min_samples': 4000},
    {'class':  4, 'min_samples': 2000},
    {'class': 12, 'min_samples':  500},
    {'class': 15, 'min_samples': 2000},
    {'class': 19, 'min_samples': 2000},
    {'class': 33, 'min_samples': 2000},
])

dfMinSamples


# Landsat variables specification

In [ ]:
# landsat short names
SATELLITE_IDS = ['l5','l7','l8']

# endemembers used to unmix data
ENDMEMBERS = {
    'l5': ENDMEMBERS_L5,
    'l7': ENDMEMBERS_L7,
    'l8': ENDMEMBERS_L8,
}

# landsat collection ids
COLLECTION_IDS = {
    'l5': 'LANDSAT/LT05/C01/T1_SR',
    'l7': 'LANDSAT/LE07/C01/T1_SR',
    'l8': 'LANDSAT/LC08/C01/T1_SR',
}

# Ancillary data
## 1. Area per classe and landsat tile

In [ ]:
# read areas table
dfAreas = pd.read_csv('../data/areas.csv')
dfAreas

## 2. Priority table

In [ ]:
# reads priority table
dfPriority = pd.read_csv('../data/priority.csv', decimal=',')

# find which tiles will be processed
dfPriority['process'] = (dfPriority['P0'] > 0.9) & (dfPriority['P0'] < 0.97)

# takes the relevant columns
dfPriority = dfPriority[['tile', 'process']]

dfPriority

## 3. Defective image list

In [ ]:
# loads image list from trash
trash = pd.read_json('../data/trash.json')

trash = list(trash.image_id.values)

## 4. Landsat tiles collection

In [ ]:
# loads ladsat tiles collection
ASSET_TILES = 'projects/mapbiomas-workspace/AUXILIAR/landsat-mask'

tilesCollection = ee.ImageCollection(ASSET_TILES)\
    .filterMetadata('version', 'equals', '2')

## 5. Output collection

In [ ]:
# output asset path
ASSET_OUTPUT = 'projects/imazon-simex/LULC/TEST/classification-2'

# output files version
OUTPUT_VERSION = '2'

# loads files from output collection
outputCollection = ee.ImageCollection(ASSET_OUTPUT)#\
    # .filterMetadata('version', 'equals', OUTPUT_VERSION)

# lists image files already saved in the collection
outputAssetIds = outputCollection.reduceColumns(
        ee.Reducer.toList(), ['system:index'])\
        .get('list')\
        .getInfo()

# list of landsat image ids to skip
skipList = map(
    lambda imageid: imageid.replace('-' + OUTPUT_VERSION, ''),
    outputAssetIds
)

skipList = list(skipList)

# Preparing data table
## 1. Legend adjustment for Amazon classes

In [ ]:
INDEX = ['year', 'tile']

dfAreas = dfAreas.replace({'class': {20: 19, 36:19, 39: 19, 41: 19}})

# aggregate areas by class, tile and year
dfAgg = dfAreas.groupby(['year', 'tile', 'class']).agg({'area': 'sum'}).reset_index()

# calculate the total area per tile and year
dfTotal = dfAreas.groupby(INDEX).agg({'area': 'sum'}).reset_index()

# merges the dfAgg with dfTotal
df = pd.merge(dfAgg, dfTotal, how="outer", on=INDEX, suffixes=(None, '_total'))

df[(df['tile'] == 226069) & (df['year'] == 2020)]

## 2. Calculate the proportion of each class

In [ ]:
df['proportion'] = df['area'].div(df['area_total'])

df[(df['tile'] == 226069) & (df['year'] == 2020)]

## 3. Number of samples based on proportions of area

In [ ]:
# calculates the number of samples based on proportions
df['n_samples'] = df['proportion'].mul(N_SAMPLES).round()

df[(df['tile'] == 226069) & (df['year'] == 2020)]

## 4. Compare the `min_samples` to `n_samples` and keep the highest value

In [ ]:
# merges minimum samples per class to data table
df = pd.merge(df, dfMinSamples, how="outer", on="class")

# replace n_samples column with the highest value betwen min_samples and n_samples
df.loc[df['min_samples'] > df['n_samples'], 'n_samples'] = df['min_samples']

df[(df['tile'] == 226069) & (df['year'] == 2020)]
    

## 5. Number of regional and global samples

In [ ]:
df['rg_samples'] = df['n_samples'].mul(P_SAMPLES['regional']).round()
df['gl_samples'] = df['n_samples'].mul(P_SAMPLES['global']).round()

df[(df['tile'] == 226069) & (df['year'] == 2020)]

In [ ]:
# merges minimum samples per class to data table
df = pd.merge(df, dfPriority, how="outer", on="tile")

df[(df['tile'] == 226069) & (df['year'] == 2020)]

# Utilitie functions

In [ ]:
def shuffle(collection, seed=1):
    """
    Adds a column of deterministic pseudorandom numbers to a collection.
    The range 0 (inclusive) to 1000000000 (exclusive).
    """

    collection = collection.randomColumn('random', seed)\
        .sort('random', True)\
        .map(
            lambda feature: feature.set(
                'new_id',
                ee.Number(feature.get('random'))
                .multiply(1000000000)
                .round()
            )
    )

    # list of random ids
    randomIdList = ee.List(
        collection.reduceColumns(ee.Reducer.toList(), ['new_id'])
        .get('list'))

    # list of sequential ids
    sequentialIdList = ee.List.sequence(1, collection.size())

    # set new ids
    shuffled = collection.remap(randomIdList, sequentialIdList, 'new_id')

    return shuffled

In [ ]:
def applyCloudAndShadowMask(collection):

    # Get cloud and shadow masks
    collectionWithMasks = getMasks(collection,
                                   cloudFlag=True,
                                   cloudScore=True,
                                   cloudShadowFlag=True,
                                   cloudShadowTdom=True,
                                   zScoreThresh=-1,
                                   shadowSumThresh=4000,
                                   dilatePixels=2,
                                   cloudHeights=ee.List.sequence(
                                       200, 10000, 500),
                                   cloudBand='cloudFlagMask')

    # collectionWithMasks = collectionWithMasks.select(specCloudBands)

    # get collection without clouds
    collectionWithoutClouds = collectionWithMasks.map(
        lambda image: image.mask(
            image.select([
                'cloudFlagMask',
                'cloudScoreMask',
                'cloudShadowFlagMask',
                'cloudShadowTdomMask'
            ]).reduce(ee.Reducer.anyNonZero()).eq(0)
        )
    )

    return collectionWithoutClouds

In [ ]:
#
SEGMENT_BANDS = ["blue", "green", "red", "nir", "swir1", "swir2"]

def getSegments(image, size=16):

    seeds = ee.Algorithms.Image.Segmentation.seedGrid(
        size=size,
        gridType='square'
    )

    snic = ee.Algorithms.Image.Segmentation.SNIC(
        image=image,
        size=size,
        compactness=1,
        connectivity=8,
        neighborhoodSize=2*size,
        seeds=seeds
    )

    snic = ee.Image(
        snic.copyProperties(image)
            .copyProperties(image, ['system:footprint'])
            .copyProperties(image, ['system:time_start']))

    return snic.select(['clusters'], ['segments'])

In [ ]:
def getSimilarMask(segments, samples):

    samplesSegments = segments.sampleRegions(
        collection=samples,
        properties=['class'],
        scale=30,
        geometries=True
    )

    segmentsValues = ee.List(
        samplesSegments
        .reduceColumns(
            ee.Reducer.toList().repeat(2),
            ['class', 'segments']
        ).get('list')
    )

    similiarMask = segments.remap(
        segmentsValues.get(1),
        segmentsValues.get(0), 0)

    return similiarMask.rename(['class'])

In [ ]:

def generateSamples(image, samples, classValues, classPoints, region, segmentsBands):

    segments = getSegments(image.select(segmentsBands), size=16)

    similarMask = getSimilarMask(segments, samples)

    similarMask = similarMask.selfMask().rename(['class'])

    newSamples = similarMask.addBands(image)\
        .stratifiedSample(
            numPoints=sum(classPoints),
            classBand='class',
            region=region,
            scale=30.0,
            classValues=classValues,
            classPoints=classPoints,
            geometries=True
    )

    return newSamples

In [ ]:
def classify(image, samples, featSpace, numberOfTrees, variablesPerSplit, minLeafPopulation):

    classifier = ee.Classifier.smileRandomForest(
        numberOfTrees=numberOfTrees,
        variablesPerSplit=variablesPerSplit,
        minLeafPopulation=minLeafPopulation
    ).train(samples, 'class', featSpace)

    return ee.Image(image
                    .select(featSpace)
                    .classify(classifier)
                    .rename(['classification'])
                    .copyProperties(image)
                    .copyProperties(image, ['system:footprint'])
                    .copyProperties(image, ['system:time_start'])
                    )

# Iterate over years and landsat tiles

In [ ]:
years = list(pd.unique(df['year']))
tiles = list(pd.unique(df[df['process'] == True]['tile']))

In [ ]:
for year in years:

    samplesName = samplesBaseName.format(year, V_SAMPLES)

    samples = ee.FeatureCollection(samplesName)\
        .filterMetadata('gv', 'not_equals', None)

    for tile in tiles:

        tileMask = tilesCollection.filterMetadata('tile', 'equals', int(tile))
        tileMask = ee.Image(tileMask.first())
        
        geometry = tileMask.geometry()
        centroid = geometry.centroid()

        dfTileYear = df[(df['tile'] == tile) & (df['year'] == year)]

        for satelliteId in SATELLITE_IDS:
            try:
                # returns a collection containing the specified parameters
                collection = getCollection(COLLECTION_IDS[satelliteId],
                                            dateStart=str(year)+'-01-01',
                                            dateEnd=str(year)+'-12-31',
                                            cloudCover=50,
                                            geometry=centroid)
                
                # drops images in trash and skip lists
                collection = collection\
                    .filter(ee.Filter.inList('system:index', skipList).Not())\
                    .filter(ee.Filter.inList('system:index', trash).Not())
            
                # returns the pattern of band names
                bands = getBandNames(satelliteId)

                # selects the images bands and rename it
                collection = collection.select(
                    bands['bandNames'],
                    bands['newNames']
                )

                # remove clouds and shadows
                collectionWithoutClouds = applyCloudAndShadowMask(collection)

                # build the feature space bands
                featureSpaceCollection = collectionWithoutClouds\
                    .map(lambda image:
                        image.addBands(srcImg=getSMAFractions(image, ENDMEMBERS[satelliteId]), overwrite=True))\
                    .map(lambda image: 
                        image.addBands(srcImg=getNDFI(image), overwrite=True))\
                    .map(lambda image: 
                        image.addBands(srcImg=getCSFI(image), overwrite=True))
                
                # lists the image ids 
                imageIds = collection.reduceColumns(
                    ee.Reducer.toList(), ['system:index'])

                imageIds = imageIds.get('list').getInfo()
                
                for imageId in imageIds:
                    
                    # step 1: get the image
                    image = featureSpaceCollection.filterMetadata(
                        'system:index', 'equals', imageId)
                    
                    image = ee.Image(image.first())

                    # step 2: select the regional samples
                    rgSamples = samples.filterBounds(geometry)
                    
                    dfTileYear['rg_samples_gee'] = dfTileYear.apply(
                        lambda serie: rgSamples
                            .filterMetadata('class', 'equals', serie['class']),
                        axis=1
                    )

                    rgSamples = ee.FeatureCollection(
                        list(dfTileYear['rg_samples_gee'].values)).flatten()

                    # step 3: generate more samples using pixel clusters (segments)
                    classValues = list(map(lambda number: int(number), dfTileYear['class'].values))
                    classPoints = list(map(lambda number: int(number), dfTileYear['rg_samples'].values))

                    rgSamplesPlus = generateSamples(
                        image=image, 
                        samples=rgSamples, 
                        classValues=classValues, 
                        classPoints=classPoints, 
                        region=geometry,
                        segmentsBands=SEGMENT_BANDS
                    )
                    
                    # step 4: select the global samples
                    shuffledSamples = shuffle(samples, seed=1)

                    dfTileYear['gl_samples_gee'] = dfTileYear.apply(
                        lambda serie: shuffledSamples
                            .filterMetadata('class', 'equals', serie['class'])
                            .limit(serie['gl_samples']),
                        axis=1
                    )

                    glSamples = ee.FeatureCollection(
                        list(dfTileYear['gl_samples_gee'].values)).flatten()

                    # step 5: merges rgSamples with glSamples
                    trainingSamples = rgSamplesPlus.merge(glSamples)

                    # step 7: image classification
                    classification = classify(
                        image=image,
                        samples=trainingSamples,
                        featSpace=FEAT_SPACE_BANDS,
                        numberOfTrees=RF_PARAMS['numberOfTrees'], 
                        variablesPerSplit=RF_PARAMS['variablesPerSplit'], 
                        minLeafPopulation=RF_PARAMS['minLeafPopulation']
                        )

                    classification = classification.updateMask(tileMask)
                    classification = classification.toByte()
                    classification = classification.set('version', OUTPUT_VERSION)

                    # step 6: export classification to gee asset
                    imageName = '{}-{}'.format(imageId, OUTPUT_VERSION)
                    
                    assetId = '{}/{}'.format(ASSET_OUTPUT, imageName)
                    
                    region = geometry.getInfo()['coordinates']

                    task = ee.batch.Export.image.toAsset(
                        image=classification,
                        description=imageName,
                        assetId=assetId,
                        pyramidingPolicy={".default": "mode"},
                        region=region,
                        scale=30,
                        # maxPixels=1e+13
                    )

                    task.start()
            except Exception as e:
                print(e)